In [8]:
import os
from dotenv import load_dotenv
from pathlib import Path

# 1. Forzar la ruta absoluta al archivo .env
ruta_actual = Path().absolute()
archivo_env = ruta_actual / ".env"

print(f"Buscando archivo en: {archivo_env}")

if archivo_env.exists():
    # El override=True fuerza a que se sobrescriba lo que haya en memoria
    load_dotenv(dotenv_path=archivo_env, override=True)
    
    api_key = os.getenv("AEMET_API_KEY")
    
    if api_key:
        # Limpieza manual por si acaso quedaron espacios invisibles
        api_key = api_key.strip()
        print(f"✅ ¡CONSEGUIDO! Clave cargada.")
        print(f"Longitud: {len(api_key)} caracteres.")
        print(f"Empieza por: {api_key[:5]}...")
    else:
        print("❌ ERROR: El archivo existe pero la variable AEMET_API_KEY no tiene valor dentro.")
else:
    print("❌ ERROR: No encuentro el archivo .env en esta carpeta.")
    print(f"Contenido de la carpeta actual: {os.listdir(ruta_actual)}")

Buscando archivo en: c:\Users\marta\OneDrive - IESE Business School\Escritorio\IRONHACK\ironhack_data\Projects\Vino Para Quedarse ML\.env
✅ ¡CONSEGUIDO! Clave cargada.
Longitud: 292 caracteres.
Empieza por: eyJhb...


In [19]:
import os
import requests
import pandas as pd
import time
from datetime import datetime
from dotenv import load_dotenv

# 1. Cargar API KEY desde el .env
load_dotenv()
API_KEY = os.getenv("AEMET_API_KEY")

# 2. Configuración de Estación y Endpoint (Peñafiel)
STATION_ID = "2166Y"
BASE_URL = "https://opendata.aemet.es/opendata/api/valores/climatologicos/mensualesanuales/datos/anioini/{year}/aniofin/{year}/estacion/{STATION_ID}"

def descargar_clima_mensual():
    headers = {"api_key": API_KEY, "cache-control": "no-cache"}
    hoy = datetime.now()
    inicio_mes_actual = datetime(hoy.year, hoy.month, 1)
    
    # Años a consultar desde 2025 en adelante
    years = range(2025, hoy.year + 1)
    dfs = []

    for y in years:
        print(f"Consultando {y}...")
        url_api = BASE_URL.format(year=y, STATION_ID=STATION_ID)
        
        try:
            # Petición a la API
            resp = requests.get(url_api, headers=headers)
            resp.raise_for_status()
            info = resp.json()

            if info.get("estado") == 200:
                # Descarga de los datos reales
                data_url = info["datos"]
                r_data = requests.get(data_url)
                r_data.raise_for_status()
                
                # AEMET puede devolver JSON o CSV, lo más común en este endpoint es JSON
                datos_raw = r_data.json()
                df_temp = pd.DataFrame(datos_raw)
                dfs.append(df_temp)
            else:
                print(f"Aviso para {y}: {info.get('descripcion')}")
            
            # Pausa para no saturar la API
            time.sleep(2)
            
        except Exception as e:
            print(f"Error descargando año {y}: {e}")

    if not dfs:
        print("No se obtuvieron datos.")
        return None

    # Concatenar y procesar
    df = pd.concat(dfs, ignore_index=True)

    # Limpieza: AEMET mete el mes '13' para el resumen anual
    df = df[df['fecha'].str.contains('-13') == False].copy()
    
    # Convertir a fecha y filtrar meses vencidos desde 2025
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df[(df['fecha'] >= '2025-01-01') & (df['fecha'] < inicio_mes_actual)].copy()

    # Mapeo de tus variables (nt_00 es el dato de heladas en este endpoint)
    columnas_map = {
        'nt_00': 'num_dias_temp_menor0',
        'p_mes': 'precipitacion_total_mensual',
        'hr': 'humedad_relativa',
        'tm_mes': 'temp_media_mes'
    }

    # Renombrar y dar formato
    df = df.rename(columns=columnas_map)
    df['Año'] = df['fecha'].dt.year
    df['Mes'] = df['fecha'].dt.month

    # Seleccionamos solo las variables solicitadas
    cols_finales = ['Año', 'Mes', 'num_dias_temp_menor0', 'precipitacion_total_mensual', 'humedad_relativa', 'temp_media_mes']
    # Filtrar solo si la columna existe (hr a veces falta si el sensor falló)
    existentes = [c for c in cols_finales if c in df.columns]
    
    df_final = df[existentes].sort_values(['Año', 'Mes']).reset_index(drop=True)
    
    return df_final

# --- Ejecución ---
df_export = descargar_clima_mensual()

if df_export is not None:
    print("\n✅ Datos obtenidos con éxito:")
    display(df_export)
    df_export.to_excel("datosaemet_mensuales_2025_en_adelante.xlsx", index=False)
    print("\n💾 Archivo guardado como: datos_mensuales_2025_en_adelante.xlsx")

Consultando 2025...
Consultando 2026...

✅ Datos obtenidos con éxito:


,Año,Mes,num_dias_temp_menor0,precipitacion_total_mensual,humedad_relativa,temp_media_mes
0,2025,1,10,54.0,79,5.9
1,2025,2,12,25.8,79,6.6
2,2025,3,8,66.8,73,7.2
3,2025,4,1,53.4,66,11.7
4,2025,5,1,40.8,59,14.2
5,2025,6,0,46.2,51,21.9
6,2025,7,0,36.8,44,22.0
7,2025,8,0,0.6,42,23.0
8,2025,9,0,0.2,52,17.4
9,2025,10,2,32.6,63,14.4



💾 Archivo guardado como: datos_mensuales_2025_en_adelante.xlsx
